In [1]:
import sys
!{sys.executable} -m pip install ftfy



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install matplotlib
!pip install nltk
!pip install numpy
!pip install pandas
!pip install regex



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import warnings
warnings.filterwarnings("ignore")
import ftfy
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re

In [4]:
from math import exp
from numpy import sign

In [5]:
!pip install scikit-learn



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install gensim



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk import PorterStemmer

In [8]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model

Reproducibility

In [9]:
np.random.seed(1234)

In [10]:
DEPRES_NROWS = 3200  # number of rows to read from DEPRESSIVE_TWEETS_CSV
RANDOM_NROWS = 4000 # number of rows to read from RANDOM_TWEETS_CSV
MAX_SEQUENCE_LENGTH = 140 # Max tweet size
MAX_NB_WORDS = 20000
EMBEDDING_DIM=300
TRAIN_SPLIT = 0.55
TEST_SPLIT = 0.25
LEARNING_RATE = 0.1
EPOCHS= 10

In [11]:
df = 'depressive_tweets_processed.csv'
RANDOM_TWEETS_CSV = 'Sentiment Analysis Dataset 2.csv'
depressive_tweets_df = pd.read_csv(df, sep = '|', header = None, usecols = range(0,9), nrows = DEPRES_NROWS)
random_tweets_df = pd.read_csv(RANDOM_TWEETS_CSV, encoding = "ISO-8859-1", usecols = range(0,3), nrows = RANDOM_NROWS)
#Embedding_file is the file which taken from this link https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&export=download
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'

In [12]:
print (depressive_tweets_df.head())

                    0           1         2                      3   
0  989292962323615744  2018-04-25  23:59:57  Eastern Standard Time  \
1  989292959844663296  2018-04-25  23:59:56  Eastern Standard Time   
2  989292951716155392  2018-04-25  23:59:54  Eastern Standard Time   
3  989292873664393218  2018-04-25  23:59:35  Eastern Standard Time   
4  989292856119472128  2018-04-25  23:59:31  Eastern Standard Time   

                 4                                                  5  6  7  8  
0         whosalli  The lack of this understanding is a small but ...  1  0  3  
1      estermnunes  i just told my parents about my depression and...  1  0  2  
2    TheAlphaAries  depression is something i don't speak about ev...  0  0  0  
3       _ojhodgson  Made myself a tortilla filled with pb&j. My de...  1  0  0  
4  DMiller96371630  @WorldofOutlaws I am gonna need depression med...  0  0  0  


In [13]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

Expand Contraction

In [14]:
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

In [15]:
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

In [16]:
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [17]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sneha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sneha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
def clean_tweets(tweets):
    cleaned_tweets = []
    for tweet in tweets:
        tweet = str(tweet)
        # if url links then dont append to avoid news articles
        # also check tweet length, save those > 10 (length of word "depression")
        if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
            # remove hashtag, @mention, emoji and image URLs
            tweet = ' '.join(
                re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", tweet).split())

            # fix weirdly encoded texts
            tweet = ftfy.fix_text(tweet)

            # expand contraction
            tweet = expandContractions(tweet)

            # remove punctuation
            tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())

            # stop words
            stop_words = set(stopwords.words('english'))
            word_tokens = nltk.word_tokenize(tweet)
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            tweet = ' '.join(filtered_sentence)

            # stemming words
            tweet = PorterStemmer().stem(tweet)
            cleaned_tweets.append(tweet)
    return cleaned_tweets
depressive_tweets_arr = [x for x in depressive_tweets_df[5]]
random_tweets_arr = [x for x in random_tweets_df['Caption']]
X_d = clean_tweets(depressive_tweets_arr)
X_r = clean_tweets(random_tweets_arr)

In [19]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_d + X_r)

In [20]:
sequences_d = tokenizer.texts_to_sequences(X_d)
sequences_r = tokenizer.texts_to_sequences(X_r)

In [21]:
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 15539 unique tokens


In [22]:
data_d = pad_sequences(sequences_d, maxlen=MAX_SEQUENCE_LENGTH)
data_r = pad_sequences(sequences_r, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data_d tensor:', data_d.shape)
print('Shape of data_r tensor:', data_r.shape)

Shape of data_d tensor: (2308, 140)
Shape of data_r tensor: (3957, 140)


In [23]:
nb_words = min(MAX_NB_WORDS, len(word_index))

In [24]:
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

In [25]:
#for (word, idx) in word_index.items():
 #
    #if word in word2vec.vocab and idx < MAX_NB_WORDS:
    #    embedding_matrix[idx] = word2vec.word_vec(word)

In [26]:
for word, idx in word_index.items():
    if word in word2vec.key_to_index and idx < MAX_NB_WORDS:
        embedding_matrix[idx] = word2vec.get_vector(word)


Assigning labels to the depressive tweets and random tweets data

In [27]:
labels_d = np.array([1] * DEPRES_NROWS)
labels_r = np.array([0] * RANDOM_NROWS)

Splitting the arrays into test (60%), validation (20%), and train data (20%)

In [28]:
perm_d = np.random.permutation(len(data_d))
idx_train_d = perm_d[:int(len(data_d)*(TRAIN_SPLIT))]
idx_test_d = perm_d[int(len(data_d)*(TRAIN_SPLIT)):int(len(data_d)*(TRAIN_SPLIT+TEST_SPLIT))]
idx_val_d = perm_d[int(len(data_d)*(TRAIN_SPLIT+TEST_SPLIT)):]

In [29]:
perm_r = np.random.permutation(len(data_r))
idx_train_r = perm_r[:int(len(data_r)*(TRAIN_SPLIT))]
idx_test_r = perm_r[int(len(data_r)*(TRAIN_SPLIT)):int(len(data_r)*(TRAIN_SPLIT+TEST_SPLIT))]
idx_val_r = perm_r[int(len(data_r)*(TRAIN_SPLIT+TEST_SPLIT)):]

Combine depressive tweets and random tweets arrays

In [30]:
data_train = np.concatenate((data_d[idx_train_d], data_r[idx_train_r]))
labels_train = np.concatenate((labels_d[idx_train_d], labels_r[idx_train_r]))
data_test = np.concatenate((data_d[idx_test_d], data_r[idx_test_r]))
labels_test = np.concatenate((labels_d[idx_test_d], labels_r[idx_test_r]))
data_val = np.concatenate((data_d[idx_val_d], data_r[idx_val_r]))
labels_val = np.concatenate((labels_d[idx_val_d], labels_r[idx_val_r]))

Shuffling

In [31]:
perm_train = np.random.permutation(len(data_train))
data_train = data_train[perm_train]
labels_train = labels_train[perm_train]
perm_test = np.random.permutation(len(data_test))
data_test = data_test[perm_test]
labels_test = labels_test[perm_test]
perm_val = np.random.permutation(len(data_val))
data_val = data_val[perm_val]
labels_val = labels_val[perm_val]

In [32]:
import numpy as np

# Verify the vocabulary size
vocab_size = len(word_index)
print(vocab_size)
# Replace 'word_index' with your actual vocabulary
embedding_dim = 100  # Replace with your desired embedding dimension

# Check the maximum index value in the training set
max_index_value_train = np.max([np.max(data_d[idx_train_d]), np.max(data_r[idx_train_r])])  # Replace 'data_train_d' and 'data_train_r' with your actual training data indices
print(max_index_value_train)
# Ensure the maximum index value is within the range of the embedding layer
if max_index_value_train >= vocab_size:
    print("Maximum index value in the training set exceeds vocabulary size")

# Create the embedding layer with the appropriate input dimensions
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

# Rest of your model definition and training code...
from keras.models import Sequential
from keras.layers import GlobalAveragePooling1D, Dense, Dropout,Flatten,Reshape

model = Sequential()
model.add(embedding_layer)
# Add other layers to your model
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
# LSTM Layer
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
# Check the maximum index value in the validation set
max_index_value_val = np.max(data_val)  # Replace 'data_val' with your actual validation data
print(max_index_value_val)
if max_index_value_val >= vocab_size:
    print ("Maximum index value in the validation set exceeds vocabulary size")

# Check the maximum index value in the test set
max_index_value_test = np.max(data_test)  # Replace 'data_test' with your actual test data
print(max_index_value_test)
if max_index_value_test >= vocab_size:
    print("Maximum index value in the test set exceeds vocabulary size")

# Continue with your validation and testing code...


15539
15539
Maximum index value in the training set exceeds vocabulary size
15532
15538


In [33]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
from keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
early_stop = EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)

# Fit the model with the defined callback
hist = model.fit(data_train, labels_train, validation_data=(data_val, labels_val), epochs=EPOCHS, batch_size=40, shuffle=True, callbacks=[early_stop])

# Plot the model accuracy
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


Epoch 1/10
61/87 [====================>.........] - ETA: 4s - loss: 0.5861 - accuracy: 0.7107

InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\sneha\myenv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\sneha\myenv\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\sneha\myenv\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\sneha\myenv\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      se

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
labels_pred = model.predict(data_test)
labels_pred = np.round(labels_pred.flatten())
accuracy = accuracy_score(labels_test, labels_pred)
print("Accuracy: %.2f%%" % (accuracy*100))

In [ ]:
print(classification_report(labels_test, labels_pred))

In [ ]:
class LogReg:
  """
  Class to represent a logistic regression model.
  """

In [ ]:
  def __init__(self, l_rate, epochs, n_features):
    """
    Create a new model with certain parameters.
    :param l_rate: Initial learning rate for model.
    :param epoch: Number of epochs to train for.
    :param n_features: Number of features.
    """
    self.l_rate = l_rate
    self.epochs = epochs
    self.coef = [0.0] * n_features
    self.bias = 0.0

In [ ]:
  def sigmoid(self, score, threshold=20.0):
    """
    Prevent overflow of exp by capping activation at 20.
    :param score: A real valued number to convert into a number between 0 and 1
    """
    if abs(score) > threshold:
      score = threshold * sign(score)
    activation = exp(score)
    return activation / (1.0 + activation)

In [ ]:
  def predict(self, features):
    """
    Given an example's features and the coefficients, predicts the class.
    :param features: List of real valued features for a single training example.
    :return: Returns the predicted class (either 0 or 1).
    """
    value = sum([features[i] * self.coef[i] for i in range(len(features))]) + self.bias
    return self.sigmoid(value)

In [ ]:
  def sg_update(self, features, label):
    """
    Computes the update to the weights based on a predicted example.
    :param features: Features to train on.
    :param label: Corresponding label for features.
    """
    yhat = self.predict(features)
    e = label - yhat
    self.bias = self.bias + self.l_rate * e * yhat * (1 - yhat)
    for i in range(len(features)):
      self.coef[i] = self.coef[i] + self.l_rate * e * yhat * (1 - yhat) * features[i]
    return

In [ ]:
  def train(self, X, y):
    """
    Computes logistic regression coefficients using stochastic gradient descent.
    :param X: Features to train on.
    :param y: Corresponding label for each set of features.
    :return: Returns a list of model weight coefficients where coef[0] is the bias.
    """
    for epoch in range(self.epochs):
      for features, label in zip(X, y):
        self.sg_update(features, label)
    return self.bias, self.coef

In [ ]:
def get_accuracy(y_bar, y_pred):
  """
  Computes what percent of the total testing data the model classified correctly.

In [ ]:
  :param y_bar: List of ground truth classes for each example.
  :param y_pred: List of model predicted class for each example.

In [ ]:
  :return: Returns a real number between 0 and 1 for the model accuracy.
    """
  correct = 0
  for i in range(len(y_bar)):
    if y_bar[i] == y_pred[i]:
      correct += 1
    accuracy = (correct / len(y_bar)) * 100.0
    return accuracy

Logistic Model

In [ ]:
logreg = LogReg(LEARNING_RATE, EPOCHS, len(data_train[0]))
bias_logreg, weights_logreg = logreg.train(data_train, labels_train)
y_logistic = [round(logreg.predict(example)) for example in data_test]

Compare accuracies

In [ ]:
accuracy_logistic = get_accuracy(y_logistic, labels_test)
print('Logistic Regression Accuracy: {:0.3f}'.format(accuracy_logistic))